In [106]:
import torch
import numpy as np
import math
import os

In [79]:
num_notes = 128
num_time_shifts = 100
num_velocities = 32
message_dim = 2*num_notes + num_velocities + num_time_shifts
instrument_numbers = [0, 6, 40, 41, 42, 43, 45, 60, 68, 70, 71, 73]

In [ ]:
###### Music Transformer version of our network ######
# Uses the relative position representation from the Music Transformer


# HuangMHA: multi-headed attention using relative position representation
# (specifically, the representation introduced by Shaw and optimized by Huang)
class HuangMHA(torch.nn.Module):
    def __init__(self, heads, embed_dim, heads):
        pass

In [ ]:
##### TransformerXL version of our network #####

In [98]:
##### Baseline Transformer version of our network #####
# Vanilla transformer, uses absolute position representation

# Taken from https://pytorch.org/tutorials/beginner/transformer_tutorial.html.
# Only change is the unsqueeze/expand in forward (accounts for multiple instruments)
class PositionalEncoding(torch.nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :].expand(-1, x.shape[1], -1)
        return self.dropout(x)
        

# EnsembleTransformer: takes a history of MIDI messages 
# for instruments in an ensemble and generates a distribution for the next message
# for a specific instrument
class EnsembleTransformer(torch.nn.Module):
    # CONSTRUCTOR
    # ARGUMENTS
    # message_dim: dimension of a MIDI message
    # embed_dim: dimension of message embedding
    # num_instruments: number of possible instruments
    # heads: number of attention heads
    # attention_layers: number of attention layers
    # ff_size: size of the feedforward output at the end of the decoder
    def __init__(self, message_dim, embed_dim, num_instruments, heads, attention_layers, ff_size):
        super(EnsembleTransformer, self).__init__()
        
        # We add the tanhed instrument embedding to each input message 
        # (this is the global conditioning idea from DeepJ, which comes from WaveNet)
        self.i_projection = torch.nn.Embedding(num_instruments, embed_dim)
        
        self.position_encoding = PositionalEncoding(embed_dim)
        
        self.embedding = torch.nn.Embedding(message_dim, embed_dim)
        
        # An encoder is used to transform histories of all instruments in the ensemble
        # except the instrument we're generating music for
        encoder_layer = torch.nn.TransformerEncoderLayer(embed_dim, heads, ff_size)
        self.encoder = torch.nn.TransformerEncoder(encoder_layer, attention_layers)
        
        # A decoder is used to transform the history of the instrument we're generating
        # music for, then combine this with the encoder output to generate the next message
        decoder_layer = torch.nn.TransformerDecoderLayer(embed_dim, heads, ff_size)
        self.decoder = torch.nn.TransformerDecoder(decoder_layer, attention_layers)
        
        self.logits = torch.nn.Linear(embed_dim, message_dim)
    
    # forward: generates a probability distribution for the next MIDI message
    # for an instrument, given the message history of the ensemble
    # ARGUMENTS
    # history: an LxN tensor, where L is the length of the longest history in
    # the ensemble, N is the number of instruments in the ensemble. As we
    # walk along the first dimension, we should see indices of MIDI events for a particular
    # instrument
    # mask: an LxN tensor, containing False where the sequence index is valid for an instrument,
    # and True elsewhere
    # gen_idx: index along the second dimension of history indicating which instrument
    # we want to generate a message for
    # RETURN: a 1xD tensor, representing the distribution for
    # the next MIDI message for each sequence. Note, to get the actual probabilities
    # you'll have to take the softmax of this tensor along dimension 1
    def forward(self, history, mask, instruments, gen_idx):
        inputs = self.embedding(history) + torch.tanh(self.i_projection(instruments)).expand(history.shape[0], 1, -1)
        
        inputs = self.position_encoding(inputs)
        
        encode_idx = np.array([i for i in range(inputs.shape[1]) if i != gen_idx])
        
        # 2nd dimension is batch size. We'll probably just use batch size 1
        encoder_inputs = inputs[:, encode_idx].view(-1, 1, inputs.shape[2])
        encoder_mask = mask[:, encode_idx].flatten().expand(encoder_inputs.shape[0])
        encoding = self.encoder(encoder_inputs, encoder_mask)
        
        decoding = self.decoder(inputs[:, gen_idx].unsqueeze(1), encoding) # Unsqueeze is for batch dimension
        
        return self.logits(decoding[-1])

In [ ]:
##### Tests for baseline transformer #####
recording = np.load('preprocessed_data/recording0.npy', allow_pickle=True)
instruments = np.load('preprocessed_data/instruments0.npy', allow_pickle=True)

history = torch.tensor(recording[0]).unsqueeze(1).long() # Test for a single instrument
instruments = torch.tensor(instrument_numbers.index(instruments[0])).unsqueeze(0).long()
mask = torch.zeros((history.shape[0], 1), dtype=torch.bool)

et = EnsembleTransformer(message_dim, 256, 12, 4, 6, 2048).double()

print(et(history, mask, instruments, 0))

In [113]:
# Custom Dataset class
class MIDIDataset(torch.utils.data.Dataset):
    # CONSTRUCTOR: creates a list of recording files and a list
    # of instrument files in root_dir. Assumes that the directory
    # contains recording0.npy to recordingN.npy,
    # as well as instruments0.npy to instrumentsN.npy
    # ARGUMENTS
    # root_dir: the directory to search
    def __init__(self, root_dir, transform=None):
        files = os.listdir(root_dir)
        self.recordings = []
        self.instruments = []
        self.times = []
        for file in files:
            if 'recording' in file:
                self.recordings.append(os.path.join(root_dir, file))
            elif 'instruments' in file:
                self.instruments.append(os.path.join(root_dir, file))
            elif 'times' in file:
                self.times.append(os.path.join(root_dir, file))
                
        assert(len(self.recordings) == len(self.instruments))
        assert(len(self.recordings) == len(self.times))
        self.recordings.sort()
        self.instruments.sort()
        self.times.sort()
        self.transform = transform

    # __len__
    # RETURN: the number of recording files in the dataset
    def __len__(self):
        return len(self.recordings)

    # __getitem__
    # RETURN: a dictionary d with keys 'instruments'
    # 'messages', and 'times'. Assuming idx is a list of indices i1, i2, i3...
    # d['instruments'][0] is a 1d numpy array containing the instrument
    # numbers for recording i1.
    # d['messages'][0] is a numpy array of numpy arrays corresponding
    # to the same file. Each inner numpy array is LxD, where L
    # is the number of MIDI messages associated with the instrument,
    # and D is the message dimension.
    # d['times'][0] is a numpy array of sequences of message times
    # for each instrument
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        message_arrs = []
        inst_arrs = []
        time_arrs = []
        
        for i in idx:
            message_arrs.append(np.load(self.recordings[i]))
            inst_arrs.append(np.load(self.intruments[i]))
            time_arrs.append(np.load(self.times[i]))
            
        sample = {'messages': message_arrs, 'instruments': inst_arrs, 'times': time_arrs}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [114]:
dataset = MIDIDataset('preprocessed_data')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

# Training - Tian

# Instantiate the model
# PyTorch boilerplate, see tutorial if needed (optimizer and dataloader intialization, stuff like that)

# DataLoader spits out one file
# For loop over instruments in file
# Generate the next message of this instrument given all previous midi messages from this instrument and ALL OTHER INSTRUMENTS

In [ ]:
# Sample from model